In [1]:
import json
from os.path import abspath
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.window import Window as W
from pyspark.sql.types import MapType,StringType,ArrayType


from datetime import datetime
from datetime import timedelta

today=datetime.now().strftime("%Y-%m-%d")
yesterday=(datetime.now()-timedelta(1)).strftime("%Y-%m-%d")

print(today,yesterday,"starting tasks at:",datetime.now())

pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = '{:.2f}'.format

warehouse_location_path = '/home/jovyan/work/spark-warehouse'

warehouse_location = abspath(warehouse_location_path)

print(warehouse_location)

spark = SparkSession \
    .builder \
    .appName("shopee-category-search-extract") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("spark.debug.maxToStringFields",200) \
    .config("spark.sql.debug.maxToStringFields",2000) \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.extraJavaOptions","-Dio.netty.tryReflectionSetAccessible=true -Xms4096m") \
    .config("spark.driver.extraJavaOptions","-Dio.netty.tryReflectionSetAccessible=true -Xms4096m") \
    .enableHiveSupport() \
    .getOrCreate()

spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")


2022-11-07 2022-11-06 starting tasks at: 2022-11-07 03:24:58.287632
/home/jovyan/work/spark-warehouse


In [2]:
import prestodb

cols=['product_no','day_has_session','last_day_has_session','total_impressions','daily_impression','clicks','users','cost',\
      'gmv','total_web_purchase','add_cart','last_7d_cost','last_7d_gmv','image_link','product_type','is_zero_suppliyers','quality_level']

def get_presto_data(sql):
    conn=prestodb.dbapi.connect(
        host='ec2-54-218-99-163.us-west-2.compute.amazonaws.com',
        port=8889,
        user='root',
        catalog='hive',
        schema='marketing',
    )
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()

    conn.close()
    return rows

In [3]:
def get_last_x_day_str(x = 1):
    return (datetime.now() - timedelta(x)).strftime('%Y-%m-%d')

last_60d = get_last_x_day_str(60)
last_7d = get_last_x_day_str(7)
last_2d = get_last_x_day_str(2)
last_1d = get_last_x_day_str(1)

sql="""
select a.*
    ,b.image_link
    ,b.product_type
    ,c.is_zero_suppliyers
    ,c.quality_level
from
(select 
    product_no,
    count(distinct dt) as day_has_session,
    max(dt) as last_day_has_session,
    sum(cast(impression as int)) as impression,
    sum(cast(impression as int))/count(distinct dt) as daily_impression,
    sum(cast(click_pv as int)) as clicks,
    sum(cast(users as int)) as users,
    sum(cast(cost as double)) as cost,
    sum(cast(web_gmv as double)) as gmv,
    sum(cast(web_purchase as int)) as total_web_purchase,
    sum(cast(add_cart_pv as int)) as add_cart,
    round(sum(case when dt >= '{last_7d}' then cast(cost as double) else 0.0 end),2)last_7d_cost,
    round(sum(case when dt >= '{last_7d}' then cast(gmv as double) else 0.0 end),2)last_7d_gmv
from marketing.ad_report_analysis_base a
where dt >= '{last_60d}'
    and ad_channel = 'facebook'
    and lower(campaign_name) not like '%_shopify_%'
    and lower(campaign_name) not like '%_deshopify_%'
    and lower(campaign_name) not like '%独立站%'
    and lower(campaign_name) like '%_msite_%'
    and cast(cost as double) > 0
    and cast(impression as int) > 0
group by 1)a
left join(
    select id,image_link,product_type
    from marketing.facebook_catalog_app_main
    where dt = '{last_2d}'
)b on a.product_no = b.id
left join(
    select pno,
        case when write_uid = 8 then 'zero-suppliyers' else 'other-source' end as is_zero_suppliyers,
        case when quality_level = 5 then 'A'
             when quality_level = 4 then 'B'
             when quality_level = 3 then 'C'
             when quality_level = 2 then 'D'
             when quality_level = 1 then 'E'
             else 'Unchecked' end as quality_level,
        count(1) cnt
    from jiayundw_dm.product_profile_df
    where date_id = '{last_1d}'
    group by 1,2,3
)c on a.product_no = c.pno
where impression>=100
order by cost desc
""".format(last_60d=last_60d,last_7d=last_7d,last_1d=last_1d,last_2d=last_2d)

print("getting data with sql:\n",sql)

rows = get_presto_data(sql)
print("how many data:", len(rows),"\n sample line:", rows[0])

rdd = spark.sparkContext.parallelize(rows)
df=rdd.toDF(cols)

df=df.withColumn("date_written", F.lit(datetime.now().strftime('%Y-%m-%d')))

df.printSchema()
df.write.format("parquet").mode("overwrite").partitionBy("date_written").save(warehouse_location_path + "/fb-data/")

getting data with sql:
 
select a.*
    ,b.image_link
    ,b.product_type
    ,c.is_zero_suppliyers
    ,c.quality_level
from
(select 
    product_no,
    count(distinct dt) as day_has_session,
    max(dt) as last_day_has_session,
    sum(cast(impression as int)) as impression,
    sum(cast(impression as int))/count(distinct dt) as daily_impression,
    sum(cast(click_pv as int)) as clicks,
    sum(cast(users as int)) as users,
    sum(cast(cost as double)) as cost,
    sum(cast(web_gmv as double)) as gmv,
    sum(cast(web_purchase as int)) as total_web_purchase,
    sum(cast(add_cart_pv as int)) as add_cart,
    round(sum(case when dt >= '2022-10-31' then cast(cost as double) else 0.0 end),2)last_7d_cost,
    round(sum(case when dt >= '2022-10-31' then cast(gmv as double) else 0.0 end),2)last_7d_gmv
from marketing.ad_report_analysis_base a
where dt >= '2022-09-08'
    and ad_channel = 'facebook'
    and lower(campaign_name) not like '%_shopify_%'
    and lower(campaign_name) not like 

In [4]:
model_expr="""
CASE WHEN total_impressions >= 5000 AND total_web_purchase <= 0 THEN 'poor-no-trans'
WHEN imp_per_purchase >= 5000 AND total_web_purchase >4 AND daily_impression >= 7000 THEN 'poor-with-trans:top'
WHEN imp_per_purchase >= 5000 AND total_web_purchase >4 AND daily_impression >= 3000 THEN 'poor-with-trans:middle'
WHEN imp_per_purchase >= 5000 AND total_web_purchase >4 THEN 'poor-with-trans:tail'
WHEN total_impressions >= 3000 
    AND (add_cart <=0 or total_impressions/add_cart >= 500)
    AND total_web_purchase <= 0 THEN 'poor-low-site-usage'
WHEN total_web_purchase > 0 AND daily_impression >= 7000 THEN 'normal-with-trans:top'
WHEN total_web_purchase > 0 THEN 'normal-with-trans:middle-tail'
ELSE 'normal-no-trans-yet' END
"""

df=df.withColumn("roi", F.expr("gmv/cost"))\
.withColumn('imp_per_purchase',F.expr('case when total_web_purchase>0 then total_impressions/total_web_purchase else 0 end'))\
.withColumn("last_7d_roi",F.expr("last_7d_gmv/last_7d_cost"))\
.withColumn('is_poor',F.expr(model_expr))

df.printSchema()

root
 |-- product_no: string (nullable = true)
 |-- day_has_session: long (nullable = true)
 |-- last_day_has_session: string (nullable = true)
 |-- total_impressions: long (nullable = true)
 |-- daily_impression: long (nullable = true)
 |-- clicks: long (nullable = true)
 |-- users: long (nullable = true)
 |-- cost: double (nullable = true)
 |-- gmv: double (nullable = true)
 |-- total_web_purchase: long (nullable = true)
 |-- add_cart: long (nullable = true)
 |-- last_7d_cost: double (nullable = true)
 |-- last_7d_gmv: double (nullable = true)
 |-- image_link: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- is_zero_suppliyers: string (nullable = true)
 |-- quality_level: string (nullable = true)
 |-- date_written: string (nullable = false)
 |-- roi: double (nullable = true)
 |-- imp_per_purchase: double (nullable = true)
 |-- last_7d_roi: double (nullable = true)
 |-- is_poor: string (nullable = false)



In [5]:
html_df=df.where("is_poor like 'poor%' and last_7d_cost >0 and product_type not like 'Shoes%'")\
.selectExpr("product_no","round(cost,2) as all_time_cost",\
            "last_7d_cost","case when last_7d_cost/cost >=0.8 then 'new-product' else 'not-new' end as is_new","roi",\
            "replace(image_link,'.jpg','_350x350.jpg') as img_link","is_zero_suppliyers","is_poor as is_poor_performence","product_type")\
.orderBy(F.desc(F.col("last_7d_cost"))).toPandas().head(500)

html_df.head(1)

from IPython.core.display import display, HTML
import re

def path_to_image_html(path):
    return '<img src="'+ path + '" width="100" loading="lazy" />'

def pno_to_link(pno):
    link = 'https://www.wholeeshopping.com/product/'+re.findall(r'\d+', pno)[0]
    return '<a href="'+ link + '" target="_blank" >' + pno + '</a>'

def prettier_category(product_type = ""):
    return '<p class="product-type">'+product_type.replace(" > ", " > <br/>")+'</p>'

def is_new_formatter(is_new = 'new-product'):
    color = 'red;'
    if is_new != 'new-product':
        color = 'green;'
    return '<span style="color:'+ color + '">' + is_new + '</span>'

html = html_df.to_html(escape=False, formatters=dict(img_link = path_to_image_html, \
                                                     product_no = pno_to_link, \
                                                     is_new = is_new_formatter, \
                                                     product_type = prettier_category))

# display(HTML(html))

/tmp/ipykernel_376/3459182417.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [6]:
from pathlib import Path
home = str(Path.home())

def table_html_to_formatted_doc(html, date = last_1d):
    title = 'Facebook Poor Performence Products - {date}'.format(date = date)
    html = """
    <!DOCTYPE html>
<html lang="en-US">
  <head>
    <title>""" + title + """</title>
    <style>
    table, th, td {
    border: 1px solid black;
    border-collapse: collapse;
    }
    th, td {
    padding: 2px 10px;
    text-align: left;
    }
    </style>
  </head>
  <body><h1>"""+title+"</h1>"+html+"</body></html>"
    
    file = home + '/work/html/fb-poor-performence-{date}.html'.format(date=date)
    print(file)

    f = open(file, "w")
    f.write(html)
    f.close()
    
    return file
    
table_html_to_formatted_doc(html)

/home/jovyan/work/html/fb-poor-performence-2022-11-06.html


'/home/jovyan/work/html/fb-poor-performence-2022-11-06.html'

In [7]:
df.groupBy("quality_level")\
.agg(F.countDistinct('product_no').alias('products'),\
     F.max("day_has_session"),\
    F.sum("total_impressions").alias("total_impressions"),\
    F.round(F.sum("gmv"),2).alias("gmv"),\
    F.round(F.sum("cost"),2).alias("cost"),\
    F.round(F.sum("last_7d_gmv"),2).alias("last_7d_gmv"),\
    F.round(F.sum("last_7d_cost"),2).alias("last_7d_cost"),\
    F.round(F.sum("gmv")/F.sum("cost"),2).alias("roi"),\
    F.round(F.sum("last_7d_gmv")/F.sum("last_7d_cost"),2).alias("last_7d_roi"))\
.withColumn("cost_%", F.round(100.00*F.col("cost")/F.sum("cost").over(W.partitionBy(F.lit(1))),2))\
.withColumn("last_7d_cost_%", F.round(100.00*F.col("last_7d_cost")/F.sum("last_7d_cost").over(W.partitionBy(F.lit(1))),2))\
.orderBy(F.desc("last_7d_cost_%")).show()

+-------------+--------+--------------------+-----------------+----------+----------+-----------+------------+----+-----------+------+--------------+
|quality_level|products|max(day_has_session)|total_impressions|       gmv|      cost|last_7d_gmv|last_7d_cost| roi|last_7d_roi|cost_%|last_7d_cost_%|
+-------------+--------+--------------------+-----------------+----------+----------+-----------+------------+----+-----------+------+--------------+
|            B|   12810|                  60|        131433425|2049764.31|1566780.08|  125023.57|    46988.62|1.31|       2.66| 45.98|         54.33|
|            C|    6277|                  60|        124731210|1774598.55| 1368442.6|   89107.02|    32298.72| 1.3|       2.76| 40.16|         37.35|
|            D|    1326|                  60|         33205577| 526064.69| 404157.71|   12588.69|     5585.26| 1.3|       2.25| 11.86|          6.46|
|    Unchecked|    4652|                  60|          5881357|   52522.6|  66040.59|     2445.8|   

In [8]:
df.groupBy("quality_level",\
           F.expr("case when is_zero_suppliyers = 'zero-suppliyers' then 'zero-suppliyers' else 'other' end as is_zero_suppliyer"))\
.agg(F.countDistinct('product_no').alias('products'),\
     F.max("day_has_session"),\
    F.sum("total_impressions").alias("total_impressions"),\
    F.round(F.sum("gmv"),2).alias("gmv"),\
    F.round(F.sum("cost"),2).alias("cost"),\
    F.round(F.sum("last_7d_gmv"),2).alias("last_7d_gmv"),\
    F.round(F.sum("last_7d_cost"),2).alias("last_7d_cost"),\
    F.round(F.sum("gmv")/F.sum("cost"),2).alias("roi"),\
    F.round(F.sum("last_7d_gmv")/F.sum("last_7d_cost"),2).alias("last_7d_roi"))\
.withColumn("cost_%", F.round(100.00*F.col("cost")/F.sum("cost").over(W.partitionBy(F.lit(1))),2))\
.withColumn("last_7d_cost_%", F.round(100.00*F.col("last_7d_cost")/F.sum("last_7d_cost").over(W.partitionBy(F.lit(1))),2))\
.orderBy(F.desc("last_7d_cost_%")).show()

+-------------+-----------------+--------+--------------------+-----------------+----------+----------+-----------+------------+----+-----------+------+--------------+
|quality_level|is_zero_suppliyer|products|max(day_has_session)|total_impressions|       gmv|      cost|last_7d_gmv|last_7d_cost| roi|last_7d_roi|cost_%|last_7d_cost_%|
+-------------+-----------------+--------+--------------------+-----------------+----------+----------+-----------+------------+----+-----------+------+--------------+
|            B|            other|   10085|                  60|        109623514|1739356.03|1293310.55|   87690.26|    32185.47|1.34|       2.72| 37.96|         37.22|
|            C|            other|    4981|                  60|        109498452|1543122.79|1183123.41|    63436.9|    23546.81| 1.3|       2.69| 34.72|         27.23|
|            B|  zero-suppliyers|    2725|                  60|         21809911| 310408.28| 273469.53|   37333.31|    14803.15|1.14|       2.52|  8.03|        

In [9]:
zero_suppliyers = df.where("is_zero_suppliyers = 'zero-suppliyers'").orderBy(F.desc("last_7d_cost")).limit(100)
pno_list = []
for index, row in zero_suppliyers.toPandas().iterrows():
    pno_list.append(row['product_no'])
    
print(pno_list)

['SWS023118402N', 'HAP023140183N', 'WSS023017455N', 'CBW023509709N', 'MSH023448306N', 'MSW024778456N', 'MEB023403576N', 'SMS022997776N', 'MJA023240729N', 'MET023069993N', 'SWS022984218N', 'MEB022932106N', 'DFS023065636N', 'SMS023001326N', 'SWS022980338N', 'CHA023652589N', 'SMS022956697N', 'DFS023039016N', 'WBL023192389N', 'WWO023169133N', 'CHK024298955N', 'CHH023725167N', 'SWS023579145N', 'MEB022928766N', 'SWS023486577N', 'CHH023908537N', 'SHS024005733N', 'MSW022952310N', 'CHH023511861N', 'SMS022924537N', 'CBA023786139N', 'ACP023550476N', 'MEA023311975N', 'SWS023028310N', 'SCS023544728N', 'CWA022983317N', 'DHC023167080N', 'WLC023480783N', 'WWK024092741N', 'BBM023051707N', 'MTT023731744N', 'CSS023021730N', 'MJA024555331N', 'CNA023290096N', 'MEB023725658N', 'BFB023759656N', 'MEB024081867N', 'MJA022991875N', 'CGP024467270N', 'LLI023014439N', 'WWL023817767N', 'MMA023006229N', 'BFB024552846N', 'SWS024165977N', 'MET023742524N', 'MBS022996925N', 'CHH023174953N', 'SWS022980822N', 'SMS022982885